In [2]:
# Bringing in the data

import pandas as pd

# Read the CSV file into a DataFrame
df_mm = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/mm_data.csv")

df_p = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/p_data.csv")

In [25]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

def multi_logistic_accuracy(df, model, strat, k, selected_features):
    """
    Custom cross-validation for predicting NBA champion using Logistic Regression.
    
    Parameters:
    - df: DataFrame containing the data.
    - model: Logistic Regression model.
    - strat: SMOTE sampling strategy.
    - k: Number of neighbors for SMOTE.
    - selected_features: List of selected features.
    
    Returns:
    - DataFrame with columns [Strategy, k_neighbors, Accuracy], sorted by Accuracy.
    """
    
    results = []

    for test_year in range(2014, 2024):
        train_data = df[df['Year'].isin(range(test_year - 5, test_year))]
        test_data = df[df['Year'] == test_year]

        X_train, y_train = train_data[selected_features], train_data['Levels']
        X_test, y_test = test_data[selected_features], test_data['Levels']

        smote = SMOTE(sampling_strategy=strat, random_state=42, k_neighbors=k)

        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

        model.fit(X_resampled, y_resampled)
        y_pred = model.predict_proba(X_test)

        # Rank teams based on predicted champion probability (Level 4)
        ranked_teams = test_data[['Team']].copy()
        ranked_teams['Score'] = y_pred[:, 4]
        ranked_teams = ranked_teams.sort_values(by='Score', ascending=False)
        predicted_champion = ranked_teams.iloc[0]['Team']
        
        # Get the actual champion team (level 4) from the test data
        actual_champion = test_data[test_data['Levels'] == 4]['Team'].values[0]

        # Convert boolean comparison to integer (1 if correct, 0 if incorrect)
        accuracy = int(predicted_champion == actual_champion)
        results.append(accuracy)

    # Compute final accuracy as a percentage
    overall_accuracy = sum(results) / len(results)

    # Store k_neighbors and strategy along with accuracy
    accuracies = pd.DataFrame({'Strategy': [strat], 'k_neighbors': [k], 'Accuracy': [overall_accuracy]})
    
    # Sort by Accuracy in descending order (higher accuracy first)
    accuracies_sorted = accuracies.sort_values(by='Accuracy', ascending=False)

    return accuracies_sorted


In [12]:
# defining the cross validation technique for our use case

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
from statsmodels.stats.outliers_influence import variance_inflation_factor

def multi_logistic_top5(df, model, strat, k, selected_features):
    """
    Custom cross-validation for predicting NBA champion using XGBoostRanker,
    with group information to rank the teams by their predicted level (score).
    """
    results = []

    # Iterate over the years you want to predict (2014 to 2023)
    for test_year in range(2014, 2024):
        
        # Training data: the previous 5 years of data (e.g., 2009-2013 for 2014 prediction)
        train_data = df[df['Year'].isin(range(test_year - 5, test_year))]
        
        # Test data: current year
        test_data = df[df['Year'] == test_year]
        
        # Prepare features and target for training and testing
        X_train = train_data[selected_features]
        y_train = train_data['Levels']

        # Fit and resample the data (X_train, y_train are your features and labels)
        smote = SMOTE(sampling_strategy=strat, random_state=42, k_neighbors=k)
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
        
        X_test = test_data[selected_features]
        y_test = test_data['Levels']

        # If more than 1 feature is selected, calculate VIF
        if len(selected_features) > 1:
            vif_data = pd.DataFrame()
            vif_data["Feature"] = X_resampled.columns
            vif_data["VIF"] = [variance_inflation_factor(X_resampled.values, i) for i in range(X_resampled.shape[1])]
            print(vif_data)
        
        # Train the model using Logistic Regression
        model.fit(X_resampled, y_resampled)
        
        # Make predictions on the test data
        y_pred = model.predict_proba(X_test)

        # Rank teams based on predicted scores
        ranked_teams = test_data[['Team']].copy()
        ranked_teams['Score'] = y_pred[:, 4]
        ranked_teams = ranked_teams.sort_values(by='Score', ascending=False)
        
        # Get the actual champion team (level 4) from the test data
        actual_champion = test_data[test_data['Levels'] == 4]['Team'].values[0]

        # Get top 5 teams and their scores
        top_teams = ranked_teams.head(5)
        top_teams_list = top_teams.apply(lambda row: f"{row['Team']}: {row['Score']:.4f}", axis=1).tolist()
        
        # Store results
        results.append([test_year, actual_champion] + top_teams_list)
    
    # Create dataframe with results
    columns = ['Year', 'Actual Champion', 'First Pick', 'Second Pick', 'Third Pick', 'Fourth Pick', 'Fifth Pick']
    results_df = pd.DataFrame(results, columns=columns)
    results_df = results_df.sort_values(by="Year", ascending=False)

    # Compute accuracy
    accuracy = (results_df['Actual Champion'] == results_df['First Pick'].str.split(': ').str[0]).mean()
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    
    return results_df

In [6]:
model1 = LogisticRegression(solver='lbfgs', max_iter=1000)

model2 = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=1000)

In [148]:
selected_features = ['W', 'eFG%']

ks = [1, 2, 3, 4]

strats = [
{0: 70, 1: 40, 2: 20, 3: 15, 4: 10},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 15, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 15, 4: 10},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 15, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 15, 4: 10},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 15, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 15, 4: 10},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 15, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 15, 4: 10},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 15, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 15, 4: 10},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 15, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 15, 4: 10},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 15, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 20, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 20, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 20, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 20, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 20, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 20, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 20, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 20, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 20, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 20, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 20, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 20, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 20, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 20, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 25, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 25, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 25, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 25, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 25, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 25, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 25, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 25, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 25, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 25, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 25, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 25, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 25, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 25, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 30, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 30, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 30, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 30, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 30, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 30, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 30, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 30, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 30, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 30, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 30, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 30, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 30, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 30, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 40, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 40, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 40, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 40, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 40, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 40, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 40, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 40, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 40, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 40, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 40, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 40, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 40, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 40, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 50, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 50, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 50, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 50, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 50, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 50, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 50, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 50, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 50, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 50, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 50, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 50, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 50, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 50, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 60, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 60, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 60, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 60, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 60, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 60, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 60, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 60, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 60, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 60, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 60, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 60, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 60, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 60, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 70, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 70, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 70, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 70, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 70, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 70, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 70, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 70, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 70, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 70, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 70, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 70, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 70, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 70, 4: 100},

{0: 70, 1: 40, 2: 20, 3: 80, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 80, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 80, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 80, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 80, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 80, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 80, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 80, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 80, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 80, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 80, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 80, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 80, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 80, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 90, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 90, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 90, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 90, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 90, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 90, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 90, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 90, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 90, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 90, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 90, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 90, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 90, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 90, 4: 100},


{0: 70, 1: 40, 2: 20, 3: 100, 4: 15},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 20},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 25},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 30},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 40},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 50},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 60},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 70},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 80},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 90},
{0: 70, 1: 40, 2: 20, 3: 100, 4: 100},

{0: 70, 1: 40, 2: 25, 3: 100, 4: 15},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 20},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 25},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 30},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 40},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 50},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 60},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 70},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 80},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 90},
{0: 70, 1: 40, 2: 25, 3: 100, 4: 100},

{0: 70, 1: 40, 2: 30, 3: 100, 4: 15},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 20},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 25},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 30},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 40},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 50},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 60},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 70},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 80},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 90},
{0: 70, 1: 40, 2: 30, 3: 100, 4: 100},

{0: 70, 1: 40, 2: 40, 3: 100, 4: 15},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 20},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 25},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 30},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 40},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 50},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 60},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 70},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 80},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 90},
{0: 70, 1: 40, 2: 40, 3: 100, 4: 100},

{0: 70, 1: 40, 2: 50, 3: 100, 4: 15},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 20},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 25},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 30},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 40},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 50},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 60},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 70},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 80},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 90},
{0: 70, 1: 40, 2: 50, 3: 100, 4: 100},

{0: 70, 1: 40, 2: 60, 3: 100, 4: 15},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 20},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 25},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 30},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 40},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 50},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 60},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 70},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 80},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 90},
{0: 70, 1: 40, 2: 60, 3: 100, 4: 100},

{0: 70, 1: 40, 2: 70, 3: 100, 4: 15},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 20},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 25},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 30},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 40},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 50},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 60},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 70},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 80},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 90},
{0: 70, 1: 40, 2: 70, 3: 100, 4: 100}]

In [150]:
#loop for model 1 on min-max data

selected_features = ['W', 'eFG%']

results1 = []

for strat in strats:
    for k in ks:
        results = multi_logistic_accuracy(df_mm, model1, strat, k, selected_features)
        results1.append(results) 

final_results1 = pd.concat(results1, ignore_index=True)

# Sort by accuracy in descending order
final_results1 = final_results1.sort_values(by="Accuracy", ascending=False)

In [156]:
final_results1.head(20)

,Strategy,k_neighbors,Accuracy
0,"{0: 70, 1: 40, 2: 20, 3: 15, 4: 10}",1,0.5
2181,"{0: 70, 1: 40, 2: 70, 3: 60, 4: 100}",2,0.5
2091,"{0: 70, 1: 40, 2: 50, 3: 60, 4: 90}",4,0.5
2092,"{0: 70, 1: 40, 2: 50, 3: 60, 4: 100}",1,0.5
2093,"{0: 70, 1: 40, 2: 50, 3: 60, 4: 100}",2,0.5
2094,"{0: 70, 1: 40, 2: 50, 3: 60, 4: 100}",3,0.5
2095,"{0: 70, 1: 40, 2: 50, 3: 60, 4: 100}",4,0.5
2103,"{0: 70, 1: 40, 2: 60, 3: 60, 4: 20}",4,0.5
2106,"{0: 70, 1: 40, 2: 60, 3: 60, 4: 25}",3,0.5
2107,"{0: 70, 1: 40, 2: 60, 3: 60, 4: 25}",4,0.5


In [160]:
#loop for model 2 on percentile data

import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

selected_features = ['W', 'eFG%']

results2 = []

for strat in strats:
    for k in ks:
        results = multi_logistic_accuracy(df_p, model2, strat, k, selected_features)
        results2.append(results) 

final_results2 = pd.concat(results2, ignore_index=True)

# Sort by accuracy in descending order
final_results2 = final_results2.sort_values(by="Accuracy", ascending=False)

In [162]:
final_results2.head(20)

,Strategy,k_neighbors,Accuracy
0,"{0: 70, 1: 40, 2: 20, 3: 15, 4: 10}",1,0.6
2797,"{0: 70, 1: 40, 2: 70, 3: 80, 4: 100}",2,0.6
2789,"{0: 70, 1: 40, 2: 70, 3: 80, 4: 80}",2,0.6
2790,"{0: 70, 1: 40, 2: 70, 3: 80, 4: 80}",3,0.6
1421,"{0: 70, 1: 40, 2: 40, 3: 40, 4: 70}",2,0.6
1420,"{0: 70, 1: 40, 2: 40, 3: 40, 4: 70}",1,0.6
1417,"{0: 70, 1: 40, 2: 40, 3: 40, 4: 60}",2,0.6
1416,"{0: 70, 1: 40, 2: 40, 3: 40, 4: 60}",1,0.6
2794,"{0: 70, 1: 40, 2: 70, 3: 80, 4: 90}",3,0.6
1413,"{0: 70, 1: 40, 2: 40, 3: 40, 4: 50}",2,0.6


In [166]:
strat = {0: 70, 1: 40, 2: 40, 3: 40, 4: 70}

k = 2

multi_logistic_top5(df_p, model2, strat, k, selected_features)

  Feature       VIF
0       W  9.620989
1    eFG%  9.620989
  Feature        VIF
0       W  12.402395
1    eFG%  12.402395
  Feature        VIF
0       W  13.090797
1    eFG%  13.090797
  Feature        VIF
0       W  12.461472
1    eFG%  12.461472
  Feature        VIF
0       W  15.790361
1    eFG%  15.790361
  Feature        VIF
0       W  15.637409
1    eFG%  15.637409
  Feature        VIF
0       W  16.085354
1    eFG%  16.085354
  Feature        VIF
0       W  14.794385
1    eFG%  14.794385
  Feature        VIF
0       W  15.989229
1    eFG%  15.989229
  Feature        VIF
0       W  12.888249
1    eFG%  12.888249
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.5318,Oklahoma City Thunder: 0.4956,Indiana Pacers: 0.4432,Milwaukee Bucks: 0.4325,Denver Nuggets: 0.3879
8,2022,Denver Nuggets,Denver Nuggets: 0.5117,Sacramento Kings: 0.4797,Boston Celtics: 0.4553,Philadelphia 76ers: 0.4077,Golden State Warriors: 0.3894
7,2021,Golden State Warriors,Golden State Warriors: 0.4813,Denver Nuggets: 0.4698,Phoenix Suns: 0.4669,Utah Jazz: 0.4633,Miami Heat: 0.4346
6,2020,Milwaukee Bucks,Brooklyn Nets: 0.4950,Phoenix Suns: 0.4692,Utah Jazz: 0.4517,Milwaukee Bucks: 0.4496,Los Angeles Clippers: 0.4386
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.5190,Utah Jazz: 0.4302,Los Angeles Lakers: 0.4298,Miami Heat: 0.4016,Toronto Raptors: 0.3653
4,2018,Toronto Raptors,Golden State Warriors: 0.5217,Milwaukee Bucks: 0.5126,Toronto Raptors: 0.4884,Houston Rockets: 0.4300,Utah Jazz: 0.3610
3,2017,Golden State Warriors,Golden State Warriors: 0.5447,Houston Rockets: 0.5346,Cleveland Cavaliers: 0.4633,Toronto Raptors: 0.4603,Philadelphia 76ers: 0.3642
2,2016,Golden State Warriors,Golden State Warriors: 0.5350,Houston Rockets: 0.4890,Cleveland Cavaliers: 0.4633,Los Angeles Clippers: 0.4199,San Antonio Spurs: 0.3638
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.5529,San Antonio Spurs: 0.5326,Cleveland Cavaliers: 0.4862,Oklahoma City Thunder: 0.4649,Los Angeles Clippers: 0.4512
0,2014,Golden State Warriors,Golden State Warriors: 0.5567,Los Angeles Clippers: 0.5318,Atlanta Hawks: 0.5253,San Antonio Spurs: 0.4580,Cleveland Cavaliers: 0.4549
